# Handwritten Digit Recognition using CNN
This notebook implements a CNN model for handwritten digit recognition using TensorFlow and Keras.

## Installing Dependencies
Run the following command if you are not using Google Colab:
```sh
pip install tensorflow numpy matplotlib pillow scikit-learn
```

In [5]:

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image
import os


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/cf/24/271e77c22724f370c24c705f394b8035b4d27e4c2c6339f3f45ab9b8258e/tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/76/ad/fa6c508a15ff79cb5409294c293388e0999b7d480f84b65e4287277434fe/tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-an

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.0.2 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.2 which is incompatible.


In [ ]:

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Resize images to 32x32
x_train_resized = np.array([tf.image.resize(img[..., np.newaxis], (32, 32)).numpy().squeeze() for img in x_train])
x_test_resized = np.array([tf.image.resize(img[..., np.newaxis], (32, 32)).numpy().squeeze() for img in x_test])

# Normalize images
x_train_resized = x_train_resized / 255.0
x_test_resized = x_test_resized / 255.0

# Split dataset
x_train, x_val, y_train, y_val = train_test_split(x_train_resized, y_train, test_size=0.2, random_state=42)

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)


In [ ]:

# Data augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2
)


In [ ]:

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(data_augmentation.flow(x_train[..., np.newaxis], y_train, batch_size=64),
          epochs=6,
          validation_data=(x_val[..., np.newaxis], y_val))


In [ ]:

# Evaluate model
test_loss, test_acc = model.evaluate(x_test_resized[..., np.newaxis], y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Save model
model.save("handwriting_cnn_model.h5")


In [ ]:

# Load the model
loaded_model = load_model("handwriting_cnn_model.h5")


In [ ]:

# Function to predict custom image
def predict_custom_image():
    file_path = input("Enter image path: ")  # Ask user to enter image path
    if not os.path.exists(file_path):
        print("File not found! Please check the path.")
        return
    
    image = Image.open(file_path).convert('L')  # Convert image to grayscale
    image = image.resize((32, 32))  # Resize to 32x32 pixels
    image = np.array(image).astype('float32') / 255.0  # Normalize pixel values
    image = image.reshape((1, 32, 32, 1))  # Reshape for model input
    prediction = loaded_model.predict(image)  # Predict digit
    print(f'Predicted digit: {np.argmax(prediction)}')

predict_custom_image()
